# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** Clearly, its a classification problem as we are trying to assign one of the two classes (1. will pass 2. may fail) to each of the students. Regression example would have been predicting a student's score.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [15]:
# Import libraries
from __future__ import division
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split


# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [16]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed']=='yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed']=='no'])

# TODO: Calculate graduation rate
grad_rate = (n_passed/(n_passed+n_failed))*100.0

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [17]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [18]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [19]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import StratifiedShuffleSplit as shuffle_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
shuffled = shuffle_split(y_all.as_matrix(), 1, None, num_train, random_state=100)
for train_index, test_index in shuffled:
    X_train, y_train = X_all.loc[train_index], y_all.loc[train_index]
    X_test, y_test = X_all.loc[test_index], y_all.loc[test_index]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** 

*1: Support Vector Classification (SVC)*

Pros: 

* SVM-SVC offers a huge set of kernel functions to utilize for classification purposes. 
* Highly memory efficient methods. Need to look at a few support vectors, once trained and predictions are quick too.

Cons: 

* Without domain knowledge it might be a long trial and error process trying out standard kernel functions and other parameters; also the SVM output parameter are not really interpretable.

* Finding the support vectors via application of the kernel function for every pair of points can be time expensive.

Why?: 

* Kernel trick makes it very easy to find non-linearity in the data if the data in the hand is not linearly separable.

* Due to pre-processing, we ended up with relative high number of features and the relatively small number of observations, near ideal setting for applying SVC.

*2: K-Nearest Neighbors*

Pros: 

* Simple, robust and highly intuitive algorithm - can be readily assesed whether the results make sense or not
* "Lazy" evaluation method as it simply memorize the data points, then when it makes predictions it has to do all of the computation

Cons: 

* Domain knowledge required to make an educated guess for the distance function.
* Highly space expensive as all the training points need to be stored.

Why?: 

* KNN is easy way to scale up training set most efficiently while training the model.
* Good idea to use it as a way to verify the results -  finding a similar student feature-set and looking at his/her pass/fail records.

*2: Random Forests*

Pros:

* To avoid overfitting, Random Forest uses multiple Decision Trees which in turn are easy to interpret and can handle both numerical and categorical data.
* Decision trees have great ability to rank the features in order of relevance, and thus require little data preparation.

Cons:

* Doesn't support online learning, models have to be recreated if new examples come up.
* Highly time expensive training process

Why?

* Random Forests can help us to evaluate the importance of features on this classification task
* Easy to interpret and humanly understandable results.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [20]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.\n".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [23]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# TODO: Initialize the three models
clf_A = SVC()
clf_B = KNeighborsClassifier()
clf_C = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=100)


# TODO: Execute the 'train_predict' function for each classifier and each training set size
for i in [100, 200, 300]:
    train_predict(clf_A, X_train[:i], y_train[:i], X_test, y_test)

for i in [100, 200, 300]:
    train_predict(clf_B, X_train[:i], y_train[:i], X_test, y_test)
              
for i in [100, 200, 300]:
    train_predict(clf_C, X_train[:i], y_train[:i], X_test, y_test)

Training a SVC using a training set size of 100. . .
Trained model in 0.0070 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 0.8690.
Made predictions in 0.0040 seconds.
F1 score for test set: 0.8101.

Training a SVC using a training set size of 200. . .
Trained model in 0.0140 seconds
Made predictions in 0.0080 seconds.
F1 score for training set: 0.8833.
Made predictions in 0.0070 seconds.
F1 score for test set: 0.8026.

Training a SVC using a training set size of 300. . .
Trained model in 0.0360 seconds
Made predictions in 0.0180 seconds.
F1 score for training set: 0.8701.
Made predictions in 0.0080 seconds.
F1 score for test set: 0.7973.

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0120 seconds.
F1 score for training set: 0.8931.
Made predictions in 0.0080 seconds.
F1 score for test set: 0.7832.

Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVC**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.004          |         0.003          |      0.8824      |      0.7746     |
| 200               |          0.005          |         0.011          |      0.8814      |      0.7838     |
| 300               |          0.025          |         0.011          |      0.8460      |      0.8333     |

** Classifer 2 - kNeighborsClassifier**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.008          |         0.008          |      0.8244      |      0.7519     |
| 200               |          0.017          |         0.011          |      0.8099      |      0.7536     |
| 300               |          0.035          |         0.015          |      0.8491      |      0.7945     |

** Classifer 3 - Random Forests**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.057          |         0.053          |      1.0000      |      0.7862     |
| 200               |          0.064          |         0.051          |      1.0000      |      0.7862     |
| 300               |          0.078          |         0.048          |      1.0000      |      0.8630     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Three models were built initially: Support Vector Machine (SVM), k Nearest Neighbors (kNN) and Random Forests classifier (RF). In brief, the kNN classifier required the least computing power, but would require more training observations in order to increase model performance. ALso, the computation time of K-NN increases a lot faster with increase in data size. The RF classifier achieved a higher performance, but required significantly more training and prediction computing time. The SVM classifier achieved a similar level of performance to the RF, but required approximately much less computing time. Further, the F1 scores for the training and testing sets were constant independent of the number of observations utilized for training, suggesting the model's performance would not decrease given less training observations.All these factors if taken together, suggests us to choose SVM as the optimal algorithem which is to be further tuned to achieve this project's objectives.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: ** In principle, a support Vector Machine (SVM) attempts to find the threshold or line (hyperplane) that maximizes the distance between the line and the closest observations of each class. This distance is called the margin. 

In our current project, the chosen SVM model involves finding the hyperplane that best separates the passing students and failing students with the maximum margin. But the features in this prediction are not distributed regularly in two part, so there is no way to draw a straight line to separate them. Here comes the advantage of SVM. The algorithm is still very much effective in classification of the complex data sets that are not linearly-separable. This is due to the fact that an SVM classifier can accept almost any equation for the boundary of classification. For instance, if data where not separable by a straight line, an SVM could find quadratic or polynomial curves. If the data is still not separable, an SVM could transform the feature data into higher dimensions and draw a high dimension classification boundary. The variety of equations (called kernels) that can be used for separation is limitless in SVM, making it highly popular classification tool. In our project, the algorithm can find a spearating hyperplane based on the student features passed and failed. E.g. students with parents who went to college, live near the school, have good health, etc will be on the "pass" side of the hyperplane and the remaining students on the "fail" side of the hyperplane. Thus when a new student comes along, SVM will see which side the new student falls on and will make a pass/fail prediction.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [22]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 
              'C':[0.01, 0.1, 0.25, 0.5, 1, 2, 5, 10,20,50], 
              'gamma':[0.001,0.01,0.1,0.5,1,2,5]}

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, greater_is_better = True, pos_label = 'yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, cv=5,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj=grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print "Best parameters to achieve this F1 score: ", grid_obj.best_params_


Made predictions in 0.0230 seconds.
Tuned model has a training F1 score of 0.9805.
Made predictions in 0.0080 seconds.
Tuned model has a testing F1 score of 0.8129.
Best parameters to achieve this F1 score:  {'kernel': 'rbf', 'C': 1, 'gamma': 0.1}


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** Tuned model has a training F1 score of 0.9805.

Tuned model has a testing F1 score of 0.8129.

Best parameters to achieve this F1 score:  {'kernel': 'rbf', 'C': 1, 'gamma': 0.1}

The training F1 score has improved a lot compared to the untuned model. Even though the testing score is marginally poor for the tuned model, I would probably attribute that to the small data we have.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.